In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import random
from random import shuffle

In [ ]:
def plotmyimage(img,title=''):
    #This function is used during the debug phase. 
    #This just plots the image 
    plt.imshow(img,cmap='gray') # draws in grayscale if it is M x N x 1, draws in color if RGB
    plt.title(title)
    plt.show()


In [ ]:
cars=glob.glob('vehicles/vehicles/*/*.png')
notcars=glob.glob('non-vehicles/non-vehicles/*/*.png')
shuffle(cars)
shuffle(notcars)

In [ ]:
#Dataset exploration and Visualization
print('Number of car images found are     :' + str(len(cars)))
print('Number of non-car images found are :' + str(len(notcars)))
print('Size of a sample image is          :' + str(mpimg.imread(cars[0]).shape))

plotmyimage(mpimg.imread(cars[random.randint(0, len(cars)-1)]),'car sample image')
plotmyimage(mpimg.imread(notcars[random.randint(0,len(notcars)-1)]),'non-car sample image')


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split


# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

def bin_spatial(img, color_space='RGB', size=(32, 32)):
    # Convert image to new color space (if specified)
    # Use cv2.resize().ravel() to create the feature vector
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)             
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(feature_image, size).ravel() 
    # Return the feature vector
    return features

# Define a function to compute color histogram features  
def color_hist(image, nbins=32, bins_range=(0, 256),vis=False):
    # Compute the histogram of the RGB channels separately
    rhist = np.histogram(image[:,:,0], bins=nbins, range=bins_range)

    ghist = np.histogram(image[:,:,1], bins=nbins, range=bins_range)

    bhist = np.histogram(image[:,:,2], bins=nbins, range=bins_range)

    # Generating bin centers
    bin_edges=rhist[1]

    bin_centers = bin_centers = (bin_edges[1:]  + bin_edges[0:len(bin_edges)-1])/2

    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((rhist[0], ghist[0], bhist[0]))
    # Return the individual histograms, bin_centers and feature vector
    if vis==True:
        return rhist, ghist, bhist, bin_centers
    else:
        return hist_features
    
    




In [ ]:
#working with HOG features
colorspace = 'RGB' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = 0 # Can be 0, 1, 2, or "ALL"



img=mpimg.imread(cars[random.randint(0, len(cars)-1)])
gray=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
features, hog_image = get_hog_features(gray, orient, pix_per_cell, cell_per_block,vis=True, feature_vec=False)
print('The function get_hog_features() resulted in features array with the size: ' +str(features.shape))
plotmyimage(img,'sample image')
plotmyimage(hog_image,'hog feature extracted image')



In [ ]:
#reading a random image
img=mpimg.imread(cars[random.randint(0, len(cars)-1)])
#plotting the sample image
plotmyimage(img,'original image')
#scaling up the pixel values so that the values are correctly represented
img=img*256
#calling function for color histofram
rh, gh, bh, bincen = color_hist(img, vis=True)
plt.bar(bincen, rh[0])
#plt.xlim(0, 256)
plt.title('R Histogram')
plt.show()

plt.bar(bincen, gh[0])
#plt.xlim(0, 256)
plt.title('G Histogram')
plt.show()

plt.bar(bincen, bh[0])
#plt.xlim(0, 256)
plt.title('B Histogram')
plt.show()

In [ ]:
img=mpimg.imread(cars[random.randint(0, len(cars)-1)])
#plotting the sample image
plotmyimage(img,'original image')
#scaling up the pixel values so that the values are correctly represented
img=img*256
plt.show()

bin_f=bin_spatial(img)
plt.plot(bin_f)
plt.title('spatial bin of the image features in RGB space')
plt.show()

bin_f=bin_spatial(img,color_space='HSV')
plt.plot(bin_f)
plt.title('spatial bin of the image features in HSV space')
plt.show()

bin_f=bin_spatial(img,color_space='LUV')
plt.plot(bin_f)
plt.title('spatial bin of the image features in LUV space')
plt.show()

bin_f=bin_spatial(img,color_space='HLS')
plt.plot(bin_f)
plt.title('spatial bin of the image features in HLS space')
plt.show()

bin_f=bin_spatial(img,color_space='YUV')
plt.plot(bin_f)
plt.title('spatial bin of the image features in YUV space')
plt.show()

bin_f=bin_spatial(img,color_space='YCrCb')
plt.plot(bin_f)
plt.title('spatial bin of the image features in YCrCb space')
plt.show()


In [ ]:
# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, 
                     cspace='RGB', 
                     orient=9, pix_per_cell=8, cell_per_block=2, 
                     hog_channel=0,spatial_size=(32,32),
                     hist_bins=32,bins_range=(0, 256)
                    ):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        image=image*256
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      
        
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        
        hist_features = color_hist(feature_image, nbins=hist_bins, bins_range=bins_range)
        
        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        # Append the new feature vector to the features list
        features.append(hog_features)
        features.append(hist_features)
        features.append(spatial_features)
    # Return list of feature vectors
    return features

In [ ]:
cars_features=extract_features(cars)
notcars_features=extract_features(notcars)
print('Total number of features of cars are     : ' +str(np.array(cars_features).shape))
print('Total number of features of non-cars are : ' +str(np.array(notcars_features).shape))

In [ ]:
X=np.hstack((cars_features,notcars_features))
X.shape
y=np.hstack((np.ones(len(cars_features)), np.zeros(len(notcars_features))))
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

In [ ]:

# Use a linear SVC
svc = LinearSVC()
# Check the training time for the SVC
svc.fit(X_train, y_train)



In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_train

In [ ]:
y_train